In [1]:
from typing import *
from datetime import datetime, timedelta, date
from core.time_utils import Bounds
from tqdm import tqdm

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np
import polars as pl

warnings.simplefilter("ignore")

<p>In this notebook we will validate that everything was calculated correctly</p>

<h4>Fetch data from Hive structure</h4>

In [3]:
bounds: Bounds = Bounds.for_days(date(2025, 1, 1), date(2025, 1, 2))
# Expand bounds such that start and end of the daily interval will not have missing values
expanded_bounds: Bounds = bounds.expand_bounds(lb_timedelta=timedelta(minutes=5), rb_timedelta=timedelta(minutes=5))

df_ticks = (
    pl.scan_parquet(r"D:\data\transformed\trades")
    .filter(
        (pl.col("date").is_between(expanded_bounds.day0, expanded_bounds.day1)) & 
        (pl.col("symbol") == "BTC-USDT") &
        (pl.col("trade_time").is_between(expanded_bounds.start_inclusive, expanded_bounds.end_exclusive))
    )
    .collect()
)

In [4]:
print(expanded_bounds)

Bounds: 2024-12-31 23:55:00 - 2025-01-02 00:04:59.999999


<h4>Make sure data is sorted by trade_time</h4>

In [5]:
df_ticks = df_ticks.sort(by="trade_time", descending=False)
assert df_ticks["trade_time"].is_sorted()

In [6]:
df_ticks = df_ticks.with_columns(
    quote_abs=pl.col("price") * pl.col("quantity"),
    side=1 - 2 * pl.col("is_buyer_maker")  # -1 if SELL, 1 if BUY
)
df_ticks = df_ticks.with_columns(
    quote_sign=pl.col("quote_abs") * pl.col("side"),
    quantity_sign=pl.col("quantity") * pl.col("side")
)

<h4>Aggregate ticks into trades on trade_time</h4>

In [7]:
df_ticks = df_ticks.sort(by="trade_time", descending=False)

df_trades: pl.DataFrame = (
    df_ticks
    .group_by("trade_time", maintain_order=True)
    .agg(
        price_first=pl.col("price").first(),  # if someone placed a trade with price impact, then price_first
        price_last=pl.col("price").last(),  # and price_last will differ
        # Amount spent in quote asset for the trade
        quote_abs=pl.col("quote_abs").sum(),
        quote_sign=pl.col("quote_sign").sum(),
        quantity_sign=pl.col("quantity_sign").sum(),
        # Amount of base asset transacted
        quantity_abs=pl.col("quantity").sum(),
        num_ticks=pl.col("price").count(),  # number of ticks for each trade
    )
)
# Create boolean indicating if the trade was long or short
df_trades = df_trades.with_columns(
    (pl.col("quote_sign") >= 0).alias("is_long")
)

df_trades

trade_time,price_first,price_last,quote_abs,quote_sign,quantity_sign,quantity_abs,num_ticks,is_long
datetime[ns],f64,f64,f64,f64,f64,f64,u32,bool
2024-12-31 23:55:00.041,93646.97,93646.97,65.552879,65.552879,0.0007,0.0007,1,true
2024-12-31 23:55:00.410,93646.97,93646.97,66.489349,66.489349,0.00071,0.00071,1,true
2024-12-31 23:55:00.975,93646.96,93646.96,320.272603,-320.272603,-0.00342,0.00342,1,false
2024-12-31 23:55:01.336,93646.97,93646.97,14.047045,14.047045,0.00015,0.00015,1,true
2024-12-31 23:55:02.135,93646.97,93646.97,999.21317,999.21317,0.01067,0.01067,1,true
…,…,…,…,…,…,…,…,…
2025-01-02 00:04:59.858464,94474.14,94474.14,86.916209,86.916209,0.00092,0.00092,1,true
2025-01-02 00:04:59.866049,94474.14,94474.14,86.916209,86.916209,0.00092,0.00092,1,true
2025-01-02 00:04:59.871885,94474.14,94474.14,86.916209,86.916209,0.00092,0.00092,1,true


In [8]:
# compute slippage
df_trades = df_trades.with_columns(
    quote_slippage_abs=(pl.col("quote_abs") - pl.col("price_first") * pl.col("quantity_abs")).abs()
)
df_trades = df_trades.with_columns(
    quote_slippage_sign=pl.col("quote_slippage_abs") * pl.col("quantity_sign").sign()
)

<h4>Add lags of time and price will help when resampling</h4>

In [9]:
df_trades = df_trades.with_columns(
    price_last_lag=pl.col("price_last").shift(1),
    trade_time_lag=pl.col("trade_time").shift(1),
    price_first_neg_lag=pl.col("price_first").shift(-1),
    trade_time_neg_lag=pl.col("trade_time").shift(-1)
)

In [13]:
sampled_features: pl.DataFrame = (
    df_trades
    .group_by_dynamic(
        index_column="trade_time", 
        every=timedelta(milliseconds=500),
        period=timedelta(seconds=5),
        closed="right",
        label="right",
        include_boundaries=True
    )
    .agg(
        asset_return=(pl.col("price_first_neg_lag").last() / pl.col("price_last_lag").first() - 1) * 1e4,
        asset_hold_time=(pl.col("trade_time_neg_lag").last() - pl.col("trade_time_lag").first()).dt.total_nanoseconds() / 1e9,
        flow_imbalance=pl.col("quote_sign").sum() / pl.col("quote_abs").sum(),
        slippage_imbalance=pl.col("quote_slippage_sign").sum() / pl.col("quote_slippage_abs").sum(),
        powerlaw_alpha= 1 + pl.len() / (pl.col("quote_abs") / pl.col("quote_abs").min()).log().sum(),
        share_of_longs=pl.col("is_long").sum() / pl.len(),
        num_agg_trades=pl.len()
    )
    .filter(
        pl.col("trade_time").is_between(bounds.start_inclusive, bounds.end_exclusive)
    )
)

In [14]:
index: pd.DatetimeIndex = pd.date_range(
    bounds.start_inclusive, bounds.end_exclusive, freq=timedelta(milliseconds=500)
)
df_index: pl.DataFrame = pl.DataFrame({"sampled_time": index})

# left join to desired time index to make sure that dimensions are correct
df_features: pl.DataFrame = df_index.join(
    sampled_features, left_on="sampled_time", right_on="trade_time", how="left"
)

<h4>Validate that everything is computed correctly</h4>

In [50]:
df_features[:, 3:]

asset_return,asset_hold_time,flow_imbalance,slippage_imbalance,powerlaw_alpha,share_of_longs,num_agg_trades
f64,f64,f64,f64,f64,f64,u32
0.0,5.163866,-0.182741,NaN,1.913531,0.5,8
0.001069,5.306116,-0.972433,-1.0,1.407413,0.428571,14
-0.001069,5.496733,-0.963204,-1.0,1.414491,0.307692,26
-0.001069,5.86163,-0.915275,-1.0,1.403956,0.366667,30
0.0,6.297712,-0.907824,-1.0,1.411564,0.363636,33
…,…,…,…,…,…,…
0.061316,6.163597,0.979475,1.0,1.277792,0.555556,27
0.061316,5.445725,0.979664,1.0,1.270401,0.576923,26
0.060259,5.439602,0.706292,1.0,1.293981,0.517241,29


In [27]:
(
    df_trades
    .filter(
        pl.col("trade_time").is_between(
            pl.datetime(2024, 12, 31, 23, 59, 55, microsecond=500_000),
            pl.datetime(2025,  1,  1,  0,  0,  0, microsecond=500_000)
        )
    )
    .select(
        (pl.col("price_first_neg_lag").last() / pl.col("price_last_lag").first() - 1) * 1e4
    )
)

price_first_neg_lag
f64
0.001069


In [33]:
timedelta(milliseconds=500).total_seconds() * 1000

500.0